# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json
from datetime import datetime
from llmchem.utils import mk_dir,clean_vram

#import clear_output

from IPython.display import clear_output

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dataset settings
n_test=50 #number of testing data
n_train_check=50 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=10 # number of reasoning data made by GPT
n_generation_iterations=100   # trial numbers to generate new self reasoning data
max_generations=50

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_3k.csv"

#project path
project_dir="results/projects/240117llama7b_10"

#reasoning options
error_threshold=30  # if abolute error is smaller than this, add to training data

In [3]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")
mk_dir(project_dir+"/train")

In [4]:
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
#load base dataset

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [6]:
#Loop: training, evaluation, data generation
for generation in range(max_generations):
    clear_output()
    print(f"Generation: {generation}")
    #prepare train dataset

    ## reason data made by GPT4
    train_dataset=copy.deepcopy(example_reasoning_dataset)

    print(f"GPT-generated reasons: {len(train_dataset)}")

    ## reason data made by model itself
    for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
        with open(path) as f:
            train_dataset.append(json.load(f))

    print(f"All-generated reasons: {len(train_dataset)}")
    random.shuffle(train_dataset)

    #train model
    clean_vram()
    model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
    train_result=train_model(model,tokenizer,train_dataset,
                    project_dir=project_dir,
                    epochs=epochs,
                    lr=lr,
                    per_device_train_batch_size=per_device_train_batch_size,
                    gradient_checkpointing=gradient_checkpointing,
                    )

    #eval

    train_check_dataset=copy.deepcopy(train_dataset[:n_train_check])
    random.shuffle(train_check_dataset)
    train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"train_{generation}"
                                )

    test_eval_result=eval_model(model,tokenizer,test_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"test_{generation}"
                                )

    #generate additional training data by self-reasoning
    self_reasoning(model,tokenizer,base_train_dataset,
                example_reasoning_dataset,project_dir,
                generation=generation,
                n_iterations=n_generation_iterations,
                error_threshold=error_threshold,
                n_max_trials=2)

Generation: 9
GPT-generated reasons: 10
All-generated reasons: 167
Using fp16 mode


Map: 100%|██████████| 167/167 [00:00<00:00, 10201.70 examples/s]


Step,Training Loss
100,1.463200
200,0.950300
300,0.795800
400,0.733800
500,0.701100


  0%|          | 0/50 [00:00<?, ?it/s]

promlem 1 / 50


...

----


##Prediction: 105.0


#Problem


 12%|█▏        | 12/100 [06:40<21:54, 14.93s/it]

----


The presence of a cyclohexyl group in the molecule can contribute to a higher melting point due to its rigidity and increased molecular weight. The cyclohexyl group can contribute a +50°C increase to the melting point. The carboxylic acid functional group typically contributes to a decrease in melting point compared to the parent compound due to less symmetry and weaker intermolecular forces, like hydrogen bonding, which might decrease the melting point by approximately -20 to -40°C.
##Prediction: 101.0


#Problem
actual:  104.0 predicted:  101.0 err:  3.0
The presence of a cyclohexyl group in the molecule can contribute to a higher melting point due to its rigidity and increased molecular weight. The cyclohexyl group can contribute a +50°C increase to the melting point. The carboxylic acid functional group typically contributes to a decrease in melting point compared to the parent compound due to less symmetry and weaker intermolecular forces, like hydrogen bonding, which might

 13%|█▎        | 13/100 [07:32<38:01, 26.22s/it]

----


The compound 3,5-dichlorophenylhydrazine has a high melting point due to its molecular structure. The presence of a phenyl group, which typically has a melting point of around 41°C, is significantly altered by the addition of a chlorine atom, which can increase the melting point due to increased polarity and hydrogen bonding. The presence of two chlorine atoms can contribute to a higher melting point due to increased molecular weight and steric hindrance. The hydrazine group can also contribute to the melting point due to its potential for hydrogen bonding and increased polarity. A rough estimate for the contribution of each chlorine atom could be +20 degrees, and the hydrazine group could contribute +20 degrees as well. The presence of two chlorine atoms and a hydrazine group can contribute to a higher melting point due to increased molecular weight and steric hindrance.
##Prediction: 105.0


#Problem
actual:  115.0 predicted:  105.0 err:  10.0
The compound 3,5-dichlorophenylhy